In [1]:
'''
!!!МЕНЯЕМ ТОЛЬКО ЭТУ ЯЧЕЙКУ!!! ДАЛЬШЕ НИЧЕГО НЕ МЕНЯЕМ !!!
PS для более тонкой настройки можно менять параметры модели в блоке обучения модели(ниже)

Блок параметров обучения модели
'''

#Количество нейронах в слоях
Conv1D_filters=10
rnn1_units = 500
lstm1_units=2000
lstm2_units=1000
lstm3_units=500
lstm4_units=250
lstm5_units=100

dense_units=50

epochs = 10

batch_size = 64


In [2]:
# Задаем дату до которой теоритически будем прогнозировать
end_future_date = '2024-12-31 23:55:00+00:00'

In [3]:
import pandas as pd

# Читаем данные на которых будем обучать модель
df_Pl = pd.read_csv('/Users/nikitasavvin/Desktop/Учеба/CNN-LSTM/cnn_lstm_notebooks/input_data/imputed_filled_P_l_LR (1).csv')


In [4]:
# Узнаем последнию известную дату. Нужна для создания промежутка значений будущих дат
end_know_date = df_Pl['time'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0] + 1


In [5]:
import numpy as np

# Создаем общий датасет с известными и будущими данными нужно для корректного общего нормирования данных

# Добавляем 5 минут чтобы последняя известная дата не попала в список для генерации будущих дат

end_know_date = pd.to_datetime(end_know_date)

# Добавление 5 минут
end_know_date = end_know_date + pd.Timedelta(minutes=5)

# Преобразование обратно в строку
end_know_date = end_know_date.strftime('%Y-%m-%d %H:%M:%S%z')

future_dates = pd.date_range(start=end_know_date, end=end_future_date, freq='5T')

future_df = pd.DataFrame({'time': future_dates, 'P_l': np.nan})

avg_pl = df_Pl['P_l'].mean()

# Заполняем Nan если они есть в датафрейме с известными данными
df_Pl['P_l'].fillna(avg_pl, inplace=True)

# Создаем общий датаферйм с известными и будущими датами для корректной нормировки данных относительно существующих и будущих дат
df_Pl_sacler = pd.concat([df_Pl, future_df], ignore_index=True)


In [6]:
# Функция нормировки дат относительно всех дат(известных и которых хотим прогнощировать)

def normalized_df(df, scaler):
# Преобразование времени в формат datetime
  df['time'] = pd.to_datetime(df['time'])
  # Создание новых столбцов
  df['year'] = df['time'].dt.year
  df['month'] = df['time'].dt.month
  df['day'] = df['time'].dt.day
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  # Удаление исходной колонки времени
  df = df.drop('time', axis=1)

  # Нормализация данных
  df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
  return df_normalized

# Обратное масштабирование данных времени
def restored_date(df_normalized, scaler):
  df_restored = pd.DataFrame(scaler.inverse_transform(df_normalized), columns=df_normalized.columns)
  df_restored['time'] = pd.to_datetime(df_restored[['year', 'month', 'day', 'hour', 'minute']])
  df_restored = df_restored.drop(['year', 'month', 'day', 'hour', 'minute'], axis=1)
  return df_restored

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Нормализация всех данных существующих и будущих
scaler = MinMaxScaler(feature_range=(0, 1))

df_Pl_normalized_all_dates = normalized_df(df_Pl_sacler, scaler)


In [8]:
# Данные для обучения отнормированные относительно всего промежутка(известных дат и дат для прогноза)

df_Pl_normalized_initial_dates = df_Pl_normalized_all_dates.loc[: end_know_date_index-1]


In [9]:
df_Pl_normalized_future_dates = df_Pl_normalized_all_dates.loc[end_know_date_index :]


In [10]:
# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values


In [11]:

# gреобразование в 5 мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)


In [12]:
from sklearn.model_selection import train_test_split

# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values

# Преобразование в 5-мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_initial_train, y_initial, test_size=0.01, random_state=42)


In [ ]:

'''
Блок обучения модели
'''
#можно менять параметры активации количество слоев и прочее

from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, concatenate
from tensorflow.keras.models import Model

# Создание входов
input_rnn = Input(shape=(1, 5), name='input_cnn')
input_lstm = Input(shape=(1, 5), name='input_lstm')

rnn1 = SimpleRNN(units=rnn1_units, activation='sigmoid')(input_rnn)
  # изменено kernel_size на 1
# pool1 = MaxPooling1D(pool_size=1)(conv1)
# flat1 = Flatten()(pool1)

lstm1 = LSTM(units=lstm1_units, activation='sigmoid', return_sequences=True)(input_lstm)
lstm2 = LSTM(units=lstm2_units, activation='sigmoid', return_sequences=True)(lstm1)
lstm3 = LSTM(units=lstm3_units, activation='sigmoid', return_sequences=True)(lstm2)
lstm4 = LSTM(units=lstm4_units, activation='sigmoid', return_sequences=True)(lstm3)
lstm5 = LSTM(units=lstm5_units, activation='sigmoid')(lstm4)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([rnn1, lstm5])

dense = Dense(units=dense_units, activation='sigmoid')(merged)

output = Dense(units=1, activation='sigmoid')(dense)

model = Model(inputs=[input_rnn, input_lstm], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Обучение модели
epochs = epochs
batch_size = batch_size
model.fit([X_train, X_train], y_train, epochs=epochs, batch_size=batch_size)


Epoch 1/10
 1536/11801 [==>...........................] - ETA: 30:10 - loss: 0.0053

In [ ]:
'''
Нейминг моделей
model_c{count}_nc{cont}_l{count}_nl{count}_e{count}_bs{count}
где c{count} - количество cnn слоев
nc{cont} -  количество нейронов cnn слоев (если несколько то через _)
l{count} - количество lstm слоев
nl{count} - количество нейронов lstm слоев (если несколько то через _)
e{count} - количество эпох обучения
bs{count} - batch_size
'''
import os


home_path_models = os.path.abspath('models')


model_name = f'model_sigmoid_rnn{1}_rn1_{rnn1_units}_l{5}_nl1_{lstm1_units}_nl2_{lstm2_units}_nl3_{lstm3_units}_e{epochs}_bs{batch_size}.h5'

model.save(f'{home_path_models}/{model_name}')


In [ ]:
model_name